In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
from keras.optimizers import Adam
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    if ((ds.Modality != 'DX') or (ds.PatientPosition not in ['AP', 'PA'])
        or (ds.BodyPartExamined != 'CHEST')):
        print('File {} is not suitable'.format(filename))
        return False
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_size): 
    # todo
#     input_img = (img - img_mean)/img_std  # I don'tuse this when preprocessing
    input_img = resize(img, img_size)
    my_idg = ImageDataGenerator(rescale=1. / 255.0,
                               featurewise_center = True,  # zero-meaning
                               featurewise_std_normalization = True)  # standardization)
    my_idg.fit(input_img)
    proc_img = my_idg.flow(input_img)
    return proc_img


# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # load json and create model
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # buid the model with the input shape
    model.build((None, 224, 224, 3))
    # load weights into new model
    model.load_weights(weight_path)
    ## Set our optimizer, loss function, and learning rate
    optimizer = Adam(lr=0.001)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model


# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo 
    pred_prob = model.predict(img)
    prediction = 'Pneumonia' if pred_prob>thresh else 'No Pneumonia'
    return prediction

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'  #path to saved model
weight_path = 'xray_class_my_model.best.hdf5'  #path to saved best weights

IMG_SIZE=(1,224,224,3)  # This might be different if you did not use vgg16
# img_mean =  # loads the mean image value they used during training preprocessing
# img_std =  # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)  #loads model
thresh = 0.4  # again, this treshold might be low but it was optimal with the training data

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is False:
        continue
        
    img_proc = preprocess_image(img,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...

No Pneumonia

Load file test2.dcm ...

No Pneumonia

Load file test3.dcm ...

No Pneumonia

Load file test4.dcm ...

File test4.dcm is not suitable

Load file test5.dcm ...

File test5.dcm is not suitable

Load file test6.dcm ...

File test6.dcm is not suitable
